In [1]:
import re
import heapq 
import copy
import sys
import math

sys.setrecursionlimit(100000)

In [2]:
x1=math.radians(30)

In [3]:
1 - math.sin(math.radians(30))

0.5

In [4]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 1.5, 1.866),
    (60,180) : (1, 1.866, 1.5),
    (90,180) : (1, 2, 1),
    (120,180) : (1, 1.866, 0.5),
    (150,180) : (1, 1.5, 0.144),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 0.5, 0.144),
    (120,0) : (1, 0.144, 0.5),
    (90,0) : (1, 0, 1),
    (60,0) : (1, 0.144, 1.5),
    (30,0) : (1, 0.5, 1.866),
    
    #(0,0) : (1,1,2)
    (30,270) : (1.5, 1, 1.866),
    (60,270) : (1.866, 1, 1.5),
    (90,270) : (2, 1, 1),
    (120,270) : (1.866, 1, 0.5),
    (150,270) : (1.5, 1, 0.144),
    #(180,180) : (1,1,0)
    (150,90) : (0.5, 1, 0.144),
    (120,90) : (0.144, 1, 0.866),
    (90,90) : (0, 1, 1),
    (60,90) : (0.144, 1, 1.5),
    (30,90) : (0.5, 1, 1.866),
    
    #(90,0) : (1,2,1)
    (90,330) : (1.5, 1.866, 1),
    (90,300) : (1.866, 1.5, 1),
    #(90,270) : (2,1,1)
    (90,240) : (1.866, 0.866, 1),
    (90,210) : (1.5, 0.5, 1),
    #(90,180) : (1,0,1)
    (90,150) : (0.5, 0.5, 1),
    (90,120) : (0.144, 0.866, 1),
    #(90,90) : (0,1,1)
    (90,60) : (0.144, 1.5, 1),
    (90,30) : (0.5, 1.866, 1)
    
}

axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    

In [5]:
heuristic((0,0), (180,180))

4.0

In [6]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("Puzzle2-10.mb")

In [7]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [8]:
gs = GlobeState(axis0_180, axis90_270, equator, 0)
#gs.RCC_axis0_180()

gs.isSolved()

False

In [9]:
gs1 = copy.deepcopy(gs)
gs1.equator == gs.equator

True

In [10]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [11]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                q = state
        print (minVal)
        
        #q.g=q.g+(2*(math.pi)/12)
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            state.g = q.g + (2*(math.pi)/12)
            f = state.g + state.getCurrentHeuristic()
            
            flag = True 
            
            if(state.g > n):
                flag=False
                
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
                    print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    if state.g < stateX.g:
                        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                        openList.remove(stateX)
                        openList.add(state)
                    flag = False
            
            if(flag==True):
                openList.add(state)
        
        #remove q from openList
        print(len(openList))
        openList.remove(q)
        closedList.add(q)
        
        
        

In [ ]:
gh = Astar(2000)

5.340930959434605
7
5.425321860374914
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
11
5.468216116647691
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
15
5.272519449130495
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
19
5.306739647182258
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
23
5.663561056813744
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
27
5.664172449854412
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
31
5.772383404354525
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
35
5.830300207899779
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
39
5.813991512664508
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
43
5.8861548149242
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
47
5.902634060562648
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
51
5.921159188757839
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
55
5.74186540383662
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
59
5.922368365774685
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
63
5.928992304150674

6.576676411696722
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
507
6.579759363010572
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
511
6.58151680891951
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
515
6.468073445528278
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
519
6.590048193762328
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
523
6.595542616195658
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
527
6.595909555159437
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
531
6.59864548939429
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
535
6.5988693705002355
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
539
6.599521801609355
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
543
6.602357130862041
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
547
6.6041709595367735
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
551
6.428717819765422
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
555
6.528300346689618
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
559
6.473017877761659
YY

6.799559065868441
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
974
6.800686011338932
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
978
6.801496303953574
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
982
6.802910676846842
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
986
6.805272308442054
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
990
6.80721684437619
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
994
6.808650924404895
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
998
6.808783342128837
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1002
6.810051144854053
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1006
6.810893813846413
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1010
6.8110238201636815
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1014
6.811711610845521
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1018
6.8133245270823615
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1022
6.798017898788039
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1026
6.8138037650

6.939684635509739
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1435
6.953598063039584
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1439
6.954063427171265
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1442
6.954286202781615
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1446
6.9544801462699954
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1450
6.955455264418794
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1454
6.956126412938595
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1458
6.95632783144289
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1462
6.957920998658307
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1465
6.959282982649361
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1469
6.9593068381291285
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1473
6.960557784937364
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1477
6.963900157089739
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1481
6.964266752062269
YYY

6.656657997312321
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1889
6.901583867091926
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1893
7.044698269207578
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1897
7.045372726094115
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1901
7.046685284087793
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1905
7.046837334112919
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1909
7.047757360944863
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1913
7.049546042799429
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1917
7.050120075801074
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1921
7.0505162621934865
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1925
7.0509272088943025
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1929
7.051625220271941
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1933
7.052425491213322
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1937
7.0530178304087805
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
1941
6.7

...

7.508164500028963
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7257
7.508200895440108
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7261
7.508330325002616
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7265
7.508640203556174
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7269
7.508748873874421
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7273
7.508911243286635
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7277
7.5089875448802506
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7281
7.508990753421701
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7285
7.509186238929876
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7288
7.1832849633662565
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7292
6.8429354536545475
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7296
7.261915350067589
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7300
7.322952640672755
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7304
7.509334156392159
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7308
7.5

7.52833238452391
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7692
7.528348017379651
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7696
7.528649542297456
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7700
7.528841785627641
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7704
7.528874007305488
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7708
7.52888136995991
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7712
7.352674914634215
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7716
7.475370430390944
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7719
7.484443466808546
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7723
7.529044459471894
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7727
7.529090178611019
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7729
7.529135356257942
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7733
7.5292371075080275
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7736
7.529410915977261
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
7740
7.53024

7.548729796809555
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8126
7.549080038232149
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8130
7.5490922038194945
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8134
7.5491606689874615
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8137
7.549585513782043
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8141
7.549826419101139
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8145
7.549977611903106
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8149
7.5502146101661705
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8153
7.550581485784357
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8157
7.550588642735812
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8161
7.098413774697719
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8165
7.5505906496472575
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8169
7.444170513733841
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8173
7.550624574534872
YYYYYYY

7.56791734262084
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8571
7.568869788994784
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8575
7.569213082657283
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8578
7.569281464345625
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8582
7.569529854359985
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8586
7.5697409341642
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8590
7.569762237182834
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8594
7.569838200350519
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8598
7.569872054519264
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8602
7.569907092271912
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8606
7.5700810828458405
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8610
7.571016789919986
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
8614
7.571043535349875
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

7.588958571578138
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9003
7.589270322533055
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9007
7.5892816151282965
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9011
7.589510840925261
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9015
7.590185146489347
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9019
7.590343616875668
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9023
7.5903931737142685
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9027
7.590404959330591
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9031
7.591088526630024
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9035
7.591226303299856
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9039
7.591329865905031
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9043
7.591357670716999
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9047
7.293477781172461
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9051
7.463492859897972
YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY
9055
7.59

In [13]:
gh.axis0_180

{(30, 180): (30, 180),
 (60, 180): (60, 180),
 (90, 180): (90, 180),
 (120, 180): (120, 180),
 (150, 180): (150, 180),
 (180, 180): (180, 180),
 (150, 0): (150, 0),
 (120, 0): (120, 0),
 (90, 0): (90, 0),
 (60, 0): (60, 0),
 (30, 0): (30, 0),
 (0, 0): (0, 0)}

In [2]:
int("inf")

ValueError: invalid literal for int() with base 10: 'inf'

In [7]:
def Astar(gs):
    k=0;
    h = []
    heapq.heappush(h, (gs.getCurrentHeuristic(), gs))
    
    while(h[0][1].isSolved() == False):    
        gs = heapq.heappop(h)[1]
        k=k+1
        print(k)
        gs1 = copy.copy(gs)
        gs2 = copy.copy(gs)
        gs3 = copy.copy(gs)
        gs4 = copy.copy(gs)
        gs5 = copy.copy(gs)
        gs6 = copy.copy(gs)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
             
        a = gs1.getCurrentHeuristic()+k
        b = gs2.getCurrentHeuristic()+k
        c = gs3.getCurrentHeuristic()+k
        d = gs4.getCurrentHeuristic()+k
        e = gs5.getCurrentHeuristic()+k
        f = gs6.getCurrentHeuristic()+k
        
        if ((a,gs1) not in h):
            heapq.heappush(h, (a, gs1))
        if ((b,gs2) not in h):
            heapq.heappush(h, (b, gs2))
        if ((c,gs3) not in h):
            heapq.heappush(h, (c, gs3))
        if ((d,gs4) not in h):
            heapq.heappush(h, (d, gs4))
        if ((e,gs5) not in h):
            heapq.heappush(h, (e, gs5))
        if ((f,gs6) not in h):
            heapq.heappush(h, (f, gs6))
        #print(type(d))
    return k

In [ ]:

    
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    